In [1]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [2]:
from models.ddpm_unet_cattn import SPVUnet
import torch
import lightning as L
from models.g_spvd import GSPVD

/opt/conda/envs/spvd/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [70]:
model_args = {
    'voxel_size' : 0.1,
    'nfs' : (32, 64, 128, 256), 
    'attn_chans' : 8, 
    'attn_start' : 3, 
    'cross_attn_chans' : 8, 
    'cross_attn_start' : 2, 
    'cross_attn_cond_dim' : 768,
}

model = SPVUnet(**model_args)
model.use_point_branch = False
model = GSPVD(model=model, lr=0.1, training_steps=1)

In [71]:
ckpt_path = '/home/ubuntu/SPVD_Lightning/checkpoints/GSPVD/all_categories_renders/checkpoints/epoch=399-step=111600.ckpt'
ckpt = torch.load(ckpt_path, weights_only=True)
model.load_state_dict(ckpt['state_dict'])

<All keys matched successfully>

In [72]:
sum(p.numel() for p in model.parameters())

25003718

In [73]:
model = model.cuda().eval()

In [74]:
from my_schedulers.ddpm_scheduler import DDPMSparseScheduler
from utils.visualization import display_pointclouds_grid

ddpm_sched = DDPMSparseScheduler(beta_min=0.0001, beta_max=0.02, steps=1000)

In [75]:
from datasets.shapenet.shapenet_loader import ShapeNet

categories = ['skateboard']
path = "../data/ShapeNet"

te = ShapeNet(path, "test", 2048, categories, load_renders=True)

Loading (test) renders for skateboard (04225987): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 424.83it/s]


In [76]:
import numpy as np

samples = 16
references = [te[idx] for idx in np.random.choice(list(range(len(te))), size=(samples,))]

In [78]:
reference_images = torch.stack([r["render-features"] for r in references]).to("cuda")

In [79]:
preds = ddpm_sched.sample(model, 16, 2048, reference=reference_images)

Sampling:  19%|██████████████████████████████                                                                                                                             | 194/1000 [00:21<01:28,  9.13it/s]


RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
display_pointclouds_grid(preds.cpu().numpy(), offset=8, point_size=0.3)

In [77]:
real = torch.stack([r["pc"] for r in references]).numpy()
display_pointclouds_grid(real, offset=8, point_size=0.3)

Output()